### Knapsack problem example
<img alt="Knapsack Problem" height="450" src="https://raw.githubusercontent.com/adamsky777/Business_Optimization/main/Pyomo_Tutorials/Assets/Knapsack%20Probblem.png" width="900"/>

In [43]:
from pyomo.environ import *

In [25]:
# 3 decision variables, (List of binary decision variables X1,X2,X3) 
products = ['Product 1', 'Product 2', 'Product 3', 'Product 4', 'Product 5']

sales_price = {
    'Product 1' : 18,
    'Product 2' : 32,
    'Product 3' : 25,
    'Product 4' : 24,
    'Product 5' : 26
}
variable_cost = {
    'Product 1' : 8,
    'Product 2' : 9,
    'Product 3' : 9,
    'Product 4' : 10,
    'Product 5' : 13
}
hours_per_unit = {
    'Product 1' : 0.3,
    'Product 2' : 0.7,
    'Product 3' : 0.5,
    'Product 4' : 0.4,
    'Product 5' : 0.4
}
demand = {
    'Product 1' : 650,
    'Product 2' : 600,
    'Product 3' : 500,
    'Product 4' : 1000,
    'Product 5' : 800
}
capacity = 700

In [27]:
model = ConcreteModel()
# Add Indexed Decision variables to the model:
model.produce = Var(products, domain=NonNegativeReals)
# Definition of the objective function:

model.profit = Objective(expr=sum((sales_price[product] - variable_cost[product]) * model.produce[product] for product in products), sense=maximize)

def capacity_cons(model):
    return sum(hours_per_unit[product] * model.produce[product] for product in products) <= capacity
model.capacity_cons = Constraint(rule=capacity_cons)


def demand_cons(model, product):
    return model.produce[product] <= demand[product]
model.demand_cons = Constraint(products, rule=demand_cons)

In [28]:
solver = SolverFactory('gurobi')
result = solver.solve(model)

In [31]:
model.display()

Model unknown

  Variables:
    produce : Size=5, Index=produce_index
        Key       : Lower : Value              : Upper : Fixed : Stale : Domain
        Product 1 :     0 :              650.0 :  None : False : False : NonNegativeReals
        Product 2 :     0 : 150.00000000000003 :  None : False : False : NonNegativeReals
        Product 3 :     0 :                0.0 :  None : False : False : NonNegativeReals
        Product 4 :     0 :             1000.0 :  None : False : False : NonNegativeReals
        Product 5 :     0 :                0.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 23950.0

  Constraints:
    capacity_cons : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 700.0 : 700.0
    demand_cons : Size=5
        Key       : Lower : Body               : Upper
        Product 1 :  None :              650.0 :  650.0
        Product 2 :  None

In [42]:
for product in products:
    print(f"Produce {round(model.produce[product](),1)} of {product}")
print(f"Total Profit: ${model.profit()}")

Produce 650.0 of Product 1
Produce 150.0 of Product 2
Produce 0.0 of Product 3
Produce 1000.0 of Product 4
Produce 0.0 of Product 5
Total Profit: $23950.0
